## 安裝套件

In [ ]:
pip install requests beautifulsoup4

## 測試網站能否爬取以及資料型態

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.hch.gov.tw/?aid=626&pid=8&page_name=sub_list&type=0&pageNo=1"
headers = {"User-Agent": "Mozilla/5.0"}

res = requests.get(url, headers=headers)
print(res.text[:])

## 爬蟲主程式

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re

# 建立資料夾儲存檔案
os.makedirs("articles_csv", exist_ok=True)

base_url = "https://www.kln.mohw.gov.tw/"
list_url = "https://www.kln.mohw.gov.tw/?aid=509&pid=114&page_name=sub_list"
headers = {"User-Agent": "Mozilla/5.0"}

res = requests.get(list_url, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

for index, row in enumerate(soup.select("tr"), start=1):
    a_tag = row.select_one("a[title]")

    if a_tag:
        title = a_tag.get_text(strip=True)
        relative_link = a_tag["href"]
        full_link = base_url + relative_link if relative_link.startswith("/") else relative_link


        # 取得文章內容
        article_res = requests.get(full_link, headers=headers)
        article_soup = BeautifulSoup(article_res.text, "html.parser")
        p_tags = article_soup.select("p")
        content = "\n".join(p.get_text(strip=True) for p in p_tags) if p_tags else "找不到內容"

        # 檔名處理：移除非法字元
        safe_title = re.sub(r'[\\/*?:"<>|]', "", title)
        filename = f"articles_csv/{index}_{safe_title}.csv"

        # 存成單一 csv
        df = pd.DataFrame([{
            "title": title,
            "url": full_link,
            "content": content
        }])
        df.to_csv(filename, encoding="utf-8-sig", index=False)

        print(f"已儲存：{filename}")

        time.sleep(5) 


已儲存：articles_csv/2_透析居家照護與社會資源.csv
已儲存：articles_csv/3_血液透析中常見症狀.csv
已儲存：articles_csv/4_動靜脈瘻管的照護.csv
已儲存：articles_csv/5_動靜脈瘻管相關問題與併發症.csv
已儲存：articles_csv/6_血鉀過高.csv
已儲存：articles_csv/7_認識血液透析相關藥物.csv
已儲存：articles_csv/8_認識透析檢驗值.csv
已儲存：articles_csv/9_血液透析病人的運動.csv
已儲存：articles_csv/10_慢性腎衰竭的治療.csv
已儲存：articles_csv/11_腎臟與腎衰竭衛教須知.csv
